In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print("numpy", np.__version__)
print("panda", pd.__version__)
print("tensorflow", tf.__version__)

2023-10-14 13:53:49.026059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


numpy 1.24.4
panda 2.0.3
tensorflow 2.11.1


In [2]:
# Load your historical Warranty Database
data = pd.read_csv('challenge2_data.csv')

In [18]:
# Assuming your CSV has columns named 'claim' and 'fm'
claims = data['claim'].values
failure_modes = data['fm'].values

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(claims)
total_words = len(tokenizer.word_index) + 1
print(tokenizer.word_index)

{'the': 1, 'brake': 2, 'and': 3, 'rear': 4, 'front': 5, 'caliper': 6, 'customer': 7, 'on': 8, 'when': 9, 'brakes': 10, 'a': 11, 'found': 12, 'noise': 13, 'fluid': 14, 'at': 15, 'was': 16, 'replaced': 17, 'for': 18, 'test': 19, 'that': 20, 'rotors': 21, 'after': 22, 'states': 23, 'leaking': 24, 'replace': 25, 'not': 26, 'clutch': 27, 'concern': 28, 'braking': 29, 'has': 30, 'check': 31, '1': 32, 'are': 33, 'there': 34, 'pads': 35, 'hose': 36, 'calipers': 37, 'leak': 38, 'time': 39, 'vibration': 40, 'this': 41, 'out': 42, '4': 43, 'before': 44, 'verified': 45, 'remove': 46, 'we': 47, 'rotor': 48, 'oil': 49, 'replacement': 50, 'ok': 51, 'i': 52, 'system': 53, '10': 54, 'gasket': 55, 'removed': 56, 'abnormal': 57, 'cover': 58, 'inspection': 59, 'r': 60, 'both': 61, '0': 62, 'by': 63, 'have': 64, 'drove': 65, 'pulsation': 66, 'sound': 67, '11': 68, 'coming': 69, 'inspected': 70, '17': 71, 's': 72, 'but': 73, 'performed': 74, 'checked': 75, 'due': 76, 'while': 77, 'he': 78, 'applied': 79, 's

In [19]:
input_sequences = []

for line in claims[0:100]:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')


In [20]:
# Define and compile the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [22]:
# Create input and output data
X = input_sequences[:, :-1] # take tokens up to but not including the last
y = input_sequences[:, -1]  # take the last token
y = y.reshape(-1, 1)

print(X)
print(y)

[[  0   0   0 ...   0   0 984]
 [  0   0   0 ...   0 984 185]
 [  0   0   0 ... 984 185   2]
 ...
 [  0   0   0 ...   3 365 175]
 [  0   0   0 ... 365 175  24]
 [  0   0   0 ... 175  24 809]]
[[185]
 [  2]
 [  6]
 ...
 [ 24]
 [809]
 [273]]


In [23]:
# Train the model
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
65/65 [==============================] - 8s 83ms/step - loss: 7.8736
Epoch 2/10
65/65 [==============================] - 5s 78ms/step - loss: 6.1308
Epoch 3/10
65/65 [==============================] - 5s 73ms/step - loss: 5.8808
Epoch 4/10
65/65 [==============================] - 5s 76ms/step - loss: 5.7630
Epoch 5/10
65/65 [==============================] - 5s 77ms/step - loss: 5.6873
Epoch 6/10
65/65 [==============================] - 5s 80ms/step - loss: 5.6363
Epoch 7/10
65/65 [==============================] - 5s 79ms/step - loss: 5.5910
Epoch 8/10
65/65 [==============================] - 5s 81ms/step - loss: 5.5475
Epoch 9/10
65/65 [==============================] - 5s 78ms/step - loss: 5.5115
Epoch 10/10
65/65 [==============================] - 5s 83ms/step - loss: 5.4634


In [15]:
seed_text = ' '  # Start with an empty string
for _ in range(15):
    sequence = tokenizer.texts_to_sequences([seed_text])[0]
    print(sequence)
    sequence = pad_sequences([sequence], maxlen=max_sequence_length-1, padding='post')
    prediction = model.predict(sequence, verbose=0)
    print(prediction)
    predicted_word_index = np.argmax(prediction, axis=-1)[0]
    print(predicted_word_index)
    predicted_word = tokenizer.index_word[predicted_word_index]
    if predicted_word == '':  # Break if the prediction is empty
        break
    seed_text += ' ' + predicted_word

print(seed_text.strip())    

[]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1, 1]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1, 1, 1]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1, 1, 1, 1]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1, 1, 1, 1, 1]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1, 1, 1, 1, 1, 1]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1, 1, 1, 1, 1, 1, 1]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1, 1, 1, 1, 1, 1, 1, 1]
[[1.4064699e-09 1.7381109e-01 1.2103095e-01 ... 9.9839848e-10
  8.4114488e-10 1.0005374e-09]]
1
[1, 1, 1, 1, 1, 1

In [ ]:
synthetic_text.append(seed_text.strip())
synthetic_classes.append(class_name)

# Create a DataFrame for synthetic data
#synthetic_data = pd.DataFrame({'claim': synthetic_text, 'fm': synthetic_classes})

# Save the synthetic data to a CSV file
#synthetic_data.to_csv('synthetic_data.csv', index=False)